# Keras vs. TensorFlow Syntax

In [3]:
import numpy as np
np.random.seed(1)

### Installation

pip install keras

pip install tensorflow

### Import

In [1]:
import keras

import tensorflow as tf

Using TensorFlow backend.


### Linear Regression

In [389]:
# data
x = np.random.randn(100)
y = 0.5 * x + 1

In [459]:
# Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation

model = Sequential()
model.add(Dense(units=1, activation="linear", input_dim=1))

model.compile(loss="mean_squared_error", optimizer="sgd")
model.summary()
model.fit(x, y, batch_size=5, validation_split=0.2, verbose=1)
model.get_weights()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 0s - loss: 0.7871 - val_loss: 0.4989
Epoch 2/10
80/80 [==============================] - 0s - loss: 0.4099 - val_loss: 0.2602
Epoch 3/10
80/80 [==============================] - 0s - loss: 0.2140 - val_loss: 0.1357
Epoch 4/10
80/80 [==============================] - 0s - loss: 0.1114 - val_loss: 0.0708
Epoch 5/10
80/80 [==============================] - 0s - loss: 0.0582 - val_loss: 0.0370
Epoch 6/10
80/80 [==============================] - 0s - loss: 0.0304 - val_loss: 0.0193
Epoch 7/10
80/80 [==============================] - 0s - loss: 0.0159 - val_loss: 0.0101
Epoch 8/1

[array([[ 0.50820935]], dtype=float32), array([ 0.96137643], dtype=float32)]

In [428]:
# http://stackoverflow.com/questions/3160699/python-progress-bar
def progressbar(it, size=30):
    count = len(it)
    def _show(_i):
        x = int(size*_i/count)
        sys.stdout.write("{}/{} [{}{}] \r".format(_i, count, "="*x, "."*(size-x)))
        sys.stdout.flush()

    _show(0)
    for i, item in enumerate(it):
        yield item
        _show(i+1)
    sys.stdout.write("\n")
    sys.stdout.flush()

In [457]:
"""
full_size = 80
batch_size = 5
-> full gradient pass = 16 (80/5)
"""
for i in progressbar(range(80)):
    if i > 16:
        # just run the progessbar without any changes
        pass
    else:
        # do something
        time.sleep(0.1)

80/80 [==============================] 


In [464]:
# TensorFlow
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Parameters
learning_rate = 0.01
training_epochs = 10
split = 0.2
batch_size = 10

graph = tf.Graph()

with graph.as_default():
    # Placeholder & Variables
    X = tf.placeholder(tf.float32, name="X")
    Y = tf.placeholder(tf.float32, name="Y")
    # tf.random_normal(shape=[]) == np.random.randn()
    W = tf.Variable(tf.random_normal(shape=[]), name="weight")
    b = tf.Variable(tf.random_normal(shape=[]), name="bias")

    # Construct a linear model
    Y_predicted = tf.add(tf.multiply(X, W), b)

    # Mean squared error
    #cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)
    cost = tf.losses.mean_squared_error(labels=Y, predictions=Y_predicted)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    

# Launch the graph
with tf.Session(graph=graph) as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph) # python -m tensorflow.tensorboard logs="./graphs"
    sess.run(init)
    
    # validation_split
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=split)

    for epoch in range(training_epochs):
        print("Epoch: {}/{}".format(epoch+1, training_epochs))
        
        train_range = round(x_train.shape[0] / batch_size)
        for index in progressbar(range(x_train.shape[0])):
            if index > train_range:
                pass
            else:
                mini_batch = np.random.choice(x_train.shape[0], batch_size, replace=False)
                _, loss = sess.run([optimizer, cost], feed_dict={X: x_train[mini_batch], Y: y_train[mini_batch]})
        
        val_range = round(x_val.shape[0] / batch_size)
        for _ in range(x_val.shape[0]): 
            mini_batch = np.random.choice(x_val.shape[0], batch_size, replace=False)
            _, val_loss = sess.run([optimizer, cost], feed_dict={X: x_val[mini_batch], Y: y_val[mini_batch]})
            
        print("loss: {} - val_loss: {}".format(loss, val_loss))
        
    print("Optimization Finished!")
    training_cost = sess.run(cost, feed_dict={X: x, Y: y})
    print("Training cost=", training_cost, "W=", sess.run(W), "b=", sess.run(b), '\n')
        
    writer.close()

Epoch: 1/10
80/80 [==============================] 
loss: 0.11840341240167618 - val_loss: 0.07780846208333969
Epoch: 2/10
80/80 [==============================] 
loss: 0.0613195076584816 - val_loss: 0.029116839170455933
Epoch: 3/10
80/80 [==============================] 
loss: 0.01843276061117649 - val_loss: 0.005032940302044153
Epoch: 4/10
80/80 [==============================] 
loss: 0.004215208347886801 - val_loss: 0.0013901116326451302
Epoch: 5/10
80/80 [==============================] 
loss: 0.0004166064318269491 - val_loss: 0.00045925291487947106
Epoch: 6/10
80/80 [==============================] 
loss: 0.00014951403136365116 - val_loss: 0.00013726545148529112
Epoch: 7/10
80/80 [==============================] 
loss: 7.649453618796542e-05 - val_loss: 3.642608135123737e-05
Epoch: 8/10
80/80 [==============================] 
loss: 1.6197009244933724e-05 - val_loss: 1.188176975119859e-05
Epoch: 9/10
80/80 [==============================] 
loss: 8.289369361591525e-06 - val_loss: 3.33

#### Notes
- Early Stopping can be implemented via ValidationMonitor but rather used TFLearn
- TFLearn on top of TF; alternative is TF-Slim

### Logistic Regression

In [503]:
# data
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [477]:
# Keras
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Dense(10, activation="softmax", input_shape=(784,)))

model.summary()

model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [478]:
model.fit(x_train, y_train, batch_size=batch_size, validation_split=0.2, verbose=1)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 0s - loss: 1.3845 - acc: 0.6689 - val_loss: 0.8994 - val_acc: 0.8246
Epoch 2/10
48000/48000 [==============================] - 0s - loss: 0.7938 - acc: 0.8276 - val_loss: 0.6604 - val_acc: 0.8584
Epoch 3/10
48000/48000 [==============================] - 0s - loss: 0.6435 - acc: 0.8508 - val_loss: 0.5637 - val_acc: 0.8703
Epoch 4/10
48000/48000 [==============================] - 0s - loss: 0.5709 - acc: 0.8608 - val_loss: 0.5102 - val_acc: 0.8792
Epoch 5/10
48000/48000 [==============================] - 0s - loss: 0.5267 - acc: 0.8685 - val_loss: 0.4756 - val_acc: 0.8834
Epoch 6/10
48000/48000 [==============================] - 0s - loss: 0.4963 - acc: 0.8730 - val_loss: 0.4511 - val_acc: 0.8871
Epoch 7/10
48000/48000 [==============================] - 0s - loss: 0.4739 - acc: 0.8778 - val_loss: 0.4330 - val_acc: 0.8893
Epoch 8/10
48000/48000 [==============================] - 0s 

In [ ]:
x_train_full, y_train_full = x_train[:1000], y_train[:1000]

In [505]:
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 128
split = 0.2

import tensorflow as tf

graph = tf.Graph()

with graph.as_default():
    X = tf.placeholder(tf.float32, shape=[None, 784])
    Y = tf.placeholder(tf.float32, shape=[None, 10])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))

    init = tf.global_variables_initializer()
    Y_predicted = tf.add(tf.matmul(X, W),  b)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    
    # evaluation
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_predicted, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
with tf.Session(graph=graph) as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph) # python -m tensorflow.tensorboard logs="./graphs"
    sess.run(init)
    
    # validation_split
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=split)

    for epoch in range(training_epochs):
        print("Epoch: {}/{}".format(epoch+1, training_epochs))
        
        train_range = round(x_train.shape[0] / batch_size)
        for index in progressbar(range(x_train.shape[0])):
            if index > train_range:
                pass
            else:
                mini_batch = np.random.choice(x_train.shape[0], batch_size, replace=False)
                _, loss, acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_train[mini_batch], Y: y_train[mini_batch]})

        
        val_range = round(x_val.shape[0] / batch_size)
        for _ in range(x_val.shape[0]): 
            mini_batch = np.random.choice(x_val.shape[0], batch_size, replace=False)
            _, val_loss, val_acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_val[mini_batch], Y: y_val[mini_batch]})
            
        print("loss: {} - acc: {} - val_loss: {} - val_acc: {}".format(loss, acc, val_loss, val_acc))

Epoch: 1/10
800/800 [==============================] 
loss: 2.243738889694214 - acc: 0.6171875 - val_loss: 0.9551129341125488 - val_acc: 0.890625
Epoch: 2/10
800/800 [==============================] 
loss: 1.046567678451538 - acc: 0.8203125 - val_loss: 0.6591847538948059 - val_acc: 0.921875
Epoch: 3/10
800/800 [==============================] 
loss: 0.9539118409156799 - acc: 0.734375 - val_loss: 0.4772654175758362 - val_acc: 0.9296875
Epoch: 4/10
800/800 [==============================] 
loss: 0.8135160803794861 - acc: 0.8125 - val_loss: 0.4343830645084381 - val_acc: 0.921875
Epoch: 5/10
800/800 [==============================] 
loss: 0.7396941781044006 - acc: 0.828125 - val_loss: 0.32984787225723267 - val_acc: 0.96875
Epoch: 6/10
800/800 [==============================] 
loss: 0.7155982255935669 - acc: 0.8046875 - val_loss: 0.28279608488082886 - val_acc: 0.9921875
Epoch: 7/10
800/800 [==============================] 
loss: 0.8052401542663574 - acc: 0.765625 - val_loss: 0.2804521620273

### Feedforward Network

In [528]:
# data
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [507]:
# Keras
batch_size = 128
num_classes = 10
epochs = 10

model = Sequential()
model.add(Dense(32, activation="relu", input_shape=(784,)))
model.add(Dense(10, activation="softmax"))

model.summary()

model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 32)                25120     
_________________________________________________________________
dense_24 (Dense)             (None, 10)                330       
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [508]:
model.fit(x_train, y_train, batch_size=batch_size, validation_split=0.2, verbose=1)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 1s - loss: 1.5283 - acc: 0.5766 - val_loss: 0.9033 - val_acc: 0.8153
Epoch 2/10
48000/48000 [==============================] - 0s - loss: 0.7361 - acc: 0.8290 - val_loss: 0.5626 - val_acc: 0.8661
Epoch 3/10
48000/48000 [==============================] - 0s - loss: 0.5407 - acc: 0.8630 - val_loss: 0.4557 - val_acc: 0.8836
Epoch 4/10
48000/48000 [==============================] - 0s - loss: 0.4622 - acc: 0.8769 - val_loss: 0.4039 - val_acc: 0.8940
Epoch 5/10
48000/48000 [==============================] - 0s - loss: 0.4189 - acc: 0.8856 - val_loss: 0.3735 - val_acc: 0.8978
Epoch 6/10
48000/48000 [==============================] - 0s - loss: 0.3906 - acc: 0.8914 - val_loss: 0.3526 - val_acc: 0.9039
Epoch 7/10
48000/48000 [==============================] - 0s - loss: 0.3705 - acc: 0.8967 - val_loss: 0.3369 - val_acc: 0.9064
Epoch 8/10
48000/48000 [==============================] - 0s 

In [509]:
x_train_full, y_train_full = x_train[:1000], y_train[:1000]

In [511]:
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 128
split = 0.2

import tensorflow as tf

graph = tf.Graph()

with graph.as_default():
    def init_weights(shape):
        weights = tf.random_normal(shape, stddev=0.1)
        return tf.Variable(weights)

    def deep_neural_network(X, w_1, w_2):
        h    = tf.nn.sigmoid(tf.matmul(X, w_1)) 
        yhat = tf.matmul(h, w_2)
        return yhat
    
    X = tf.placeholder(tf.float32, shape=[None, 784])
    Y = tf.placeholder(tf.float32, shape=[None, 10])
    w_1 = init_weights((784, 32))
    w_2 = init_weights((32, 10))

    init = tf.global_variables_initializer()
    Y_predicted = deep_neural_network(X, w_1, w_2)

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_predicted))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
    
    # evaluation
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_predicted, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
with tf.Session(graph=graph) as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph) # python -m tensorflow.tensorboard logs="./graphs"
    sess.run(init)
    
    # validation_split
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=split)

    for epoch in range(training_epochs):
        print("Epoch: {}/{}".format(epoch+1, training_epochs))
        
        train_range = round(x_train.shape[0] / batch_size)
        for index in progressbar(range(x_train.shape[0])):
            if index > train_range:
                pass
            else:
                mini_batch = np.random.choice(x_train.shape[0], batch_size, replace=False)
                _, loss, acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_train[mini_batch], Y: y_train[mini_batch]})

        
        val_range = round(x_val.shape[0] / batch_size)
        for _ in range(x_val.shape[0]): 
            mini_batch = np.random.choice(x_val.shape[0], batch_size, replace=False)
            _, val_loss, val_acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_val[mini_batch], Y: y_val[mini_batch]})
            
        print("loss: {} - acc: {} - val_loss: {} - val_acc: {}".format(loss, acc, val_loss, val_acc))

Epoch: 1/10
800/800 [==============================] 
loss: 2.357222318649292 - acc: 0.1484375 - val_loss: 2.2194454669952393 - val_acc: 0.28125
Epoch: 2/10
800/800 [==============================] 
loss: 2.257216453552246 - acc: 0.2421875 - val_loss: 2.149240016937256 - val_acc: 0.4140625
Epoch: 3/10
800/800 [==============================] 
loss: 2.1923890113830566 - acc: 0.359375 - val_loss: 2.0849528312683105 - val_acc: 0.4140625
Epoch: 4/10
800/800 [==============================] 
loss: 2.0928196907043457 - acc: 0.40625 - val_loss: 1.9503047466278076 - val_acc: 0.546875
Epoch: 5/10
800/800 [==============================] 
loss: 2.0281476974487305 - acc: 0.453125 - val_loss: 1.8639016151428223 - val_acc: 0.6796875
Epoch: 6/10
800/800 [==============================] 
loss: 1.9074482917785645 - acc: 0.625 - val_loss: 1.762959599494934 - val_acc: 0.671875
Epoch: 7/10
800/800 [==============================] 
loss: 1.7573871612548828 - acc: 0.6484375 - val_loss: 1.6267309188842773 -

### Convolutional Neural Network

In [529]:
# data
from keras.datasets import mnist

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((-1, 28, 28, 1))
x_train = x_train.astype("float32")
x_train /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [516]:
# Keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

model = Sequential()
model.add(Conv2D(32, (5, 5), strides=(1, 1), activation="relu", input_shape=(28, 28, 1), padding="same")) # same -> input = output
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Conv2D(64, (5, 5), strides=(1, 1), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="sgd")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_27 (Dense)             (None, 1024)              3212288   
_________________________________________________________________
dense_28 (Dense)             (None, 10)                10250     
Total para

In [519]:
model.fit(x_train[:1000], y_train[:1000], batch_size=32, epochs=10, validation_split=0.2, verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 1s - loss: 1.9131 - val_loss: 1.7521ss: 1.
Epoch 2/10
800/800 [==============================] - 1s - loss: 1.5303 - val_loss: 1.3653
Epoch 3/10
800/800 [==============================] - 1s - loss: 1.1207 - val_loss: 0.9838
Epoch 4/10
800/800 [==============================] - 2s - loss: 0.8032 - val_loss: 0.7448
Epoch 5/10
800/800 [==============================] - 2s - loss: 0.6500 - val_loss: 0.6654
Epoch 6/10
800/800 [==============================] - 2s - loss: 0.5567 - val_loss: 0.6286
Epoch 7/10
800/800 [==============================] - 2s - loss: 0.5007 - val_loss: 0.5702
Epoch 8/10
800/800 [==============================] - 2s - loss: 0.4536 - val_loss: 0.5901
Epoch 9/10
800/800 [==============================] - 2s - loss: 0.4886 - val_loss: 0.5311
Epoch 10/10
800/800 [==============================] - 2s - loss: 0.3710 - val_loss: 0.5424


In [530]:
x_train_full = x_train[:1000]
y_train_full = y_train[:1000]

In [531]:
# TensorFlow
import tensorflow as tf

learning_rate = 0.01
training_epochs = 10
batch_size = 32

graph = tf.Graph()

with graph.as_default():
    X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
    Y = tf.placeholder(tf.float32, shape=[None, 10])

    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

    def max_pool_2x2(x):
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])

    h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7 * 7 * 64, 1024]) # need to understand how paddle and stride works to define the numbers
    b_fc1 = bias_variable([1024])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    Y_predicted = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

    init = tf.global_variables_initializer()
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(Y_predicted), reduction_indices=[1]))
    optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

    correct_prediction = tf.equal(tf.argmax(Y_predicted, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
with tf.Session(graph=graph) as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph) # python -m tensorflow.tensorboard logs="./graphs"
    sess.run(init)
    
    # validation_split
    x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=split)

    for epoch in range(training_epochs):
        print("Epoch: {}/{}".format(epoch+1, training_epochs))
        
        train_range = round(x_train.shape[0] / batch_size)
        for index in progressbar(range(x_train.shape[0])):
            if index > train_range:
                pass
            else:
                mini_batch = np.random.choice(x_train.shape[0], batch_size, replace=False)
                _, loss, acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_train[mini_batch], Y: y_train[mini_batch]})

        
        val_range = round(x_val.shape[0] / batch_size)
        for _ in range(x_val.shape[0]): 
            mini_batch = np.random.choice(x_val.shape[0], batch_size, replace=False)
            _, val_loss, val_acc = sess.run([optimizer, cross_entropy, accuracy], feed_dict={X: x_val[mini_batch], Y: y_val[mini_batch]})
            
        print("loss: {} - acc: {} - val_loss: {} - val_acc: {}".format(loss, acc, val_loss, val_acc))

Epoch: 1/10
800/800 [==============================] 
loss: 1.0250860452651978 - acc: 0.6875 - val_loss: 0.016407014802098274 - val_acc: 1.0
Epoch: 2/10
800/800 [==============================] 
loss: 0.22624161839485168 - acc: 0.90625 - val_loss: 0.00957321748137474 - val_acc: 1.0
Epoch: 3/10
800/800 [==============================] 
loss: 0.19930750131607056 - acc: 0.96875 - val_loss: 0.004512276966124773 - val_acc: 1.0
Epoch: 4/10
800/800 [==============================] 
loss: 0.07598122954368591 - acc: 1.0 - val_loss: 0.003296941053122282 - val_acc: 1.0
Epoch: 5/10
800/800 [==============================] 
loss: 0.17946262657642365 - acc: 0.9375 - val_loss: 0.0025500478222966194 - val_acc: 1.0
Epoch: 6/10
800/800 [==============================] 
loss: 0.036343879997730255 - acc: 1.0 - val_loss: 0.0017508012242615223 - val_acc: 1.0
Epoch: 7/10
800/800 [==============================] 
loss: 0.11032424122095108 - acc: 0.96875 - val_loss: 0.0011565235909074545 - val_acc: 1.0
Epoch: 